# 📊 Batch Analysis with FragMentor

This notebook demonstrates how to analyze multiple samples efficiently and compare results.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from fragmentomics import analyze_sizes
from fragmentomics.io import BamReader

## Analyzing Multiple BAM Files

### Method 1: Python Loop

In [ ]:
# Example: Analyze multiple BAM files
def analyze_bam_batch(bam_paths):
    """Analyze multiple BAM files and return a DataFrame."""
    results = []
    
    for path in bam_paths:
        path = Path(path)
        print(f"Processing {path.name}...")
        
        # Read sizes from BAM
        reader = BamReader(path)
        sizes = reader.extract_sizes()
        
        # Analyze
        dist = analyze_sizes(sizes)
        
        # Collect results
        results.append({
            "sample": path.stem,
            "n_fragments": dist.n_fragments,
            "mean_size": dist.mean,
            "median_size": dist.median,
            "ratio_short": dist.ratio_short,
            "ratio_mono": dist.ratio_mono,
            "ratio_di": dist.ratio_di,
            "peak_mono": dist.peak_mono,
            "periodicity": dist.periodicity_10bp,
        })
    
    return pd.DataFrame(results)

# Demo with synthetic data
print("For real analysis, use:")
print('df = analyze_bam_batch(["sample1.bam", "sample2.bam", ...])')

### Method 2: Command Line (Recommended for Large Datasets)

```bash
# Process multiple files in parallel
fragmentomics batch samples/*.bam -o results/ --threads 8

# Or analyze with a BED file of regions
fragmentomics sizes sample.bam --bed regions.bed -o output/
```

## Working with Results

In [ ]:
# Simulate batch results for demonstration
np.random.seed(42)

n_samples = 20
df = pd.DataFrame({
    "sample": [f"sample_{i:02d}" for i in range(n_samples)],
    "group": ["healthy"] * 10 + ["cancer"] * 10,
    "n_fragments": np.random.randint(50000, 200000, n_samples),
    "mean_size": np.concatenate([
        np.random.normal(165, 5, 10),  # Healthy
        np.random.normal(145, 8, 10),  # Cancer
    ]),
    "ratio_short": np.concatenate([
        np.random.beta(2, 10, 10),  # Healthy: low
        np.random.beta(5, 6, 10),   # Cancer: high
    ]),
    "ratio_mono": np.concatenate([
        np.random.beta(8, 3, 10),   # Healthy: high
        np.random.beta(5, 5, 10),   # Cancer: moderate
    ]),
})

df.head()

In [ ]:
# Summary statistics by group
df.groupby("group").agg({
    "mean_size": ["mean", "std"],
    "ratio_short": ["mean", "std"],
    "ratio_mono": ["mean", "std"],
}).round(3)

In [ ]:
# Visualize group differences
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

for ax, col, title in zip(
    axes,
    ["mean_size", "ratio_short", "ratio_mono"],
    ["Mean Fragment Size", "Short Fragment Ratio", "Mononucleosome Ratio"]
):
    for group, color in [("healthy", "#2ecc71"), ("cancer", "#e74c3c")]:
        data = df[df["group"] == group][col]
        ax.hist(data, alpha=0.7, label=group, color=color, bins=8)
    ax.set_xlabel(title)
    ax.legend()

plt.tight_layout()
plt.show()

## Exporting Results

In [ ]:
# Export to CSV
# df.to_csv("batch_results.csv", index=False)

# Export to Excel with multiple sheets
# with pd.ExcelWriter("batch_results.xlsx") as writer:
#     df.to_excel(writer, sheet_name="All Samples", index=False)
#     df.groupby("group").mean().to_excel(writer, sheet_name="Group Means")

print("Export options:")
print("  df.to_csv('results.csv')")
print("  df.to_parquet('results.parquet')")
print("  df.to_excel('results.xlsx')")

## Parallel Processing

For large cohorts, use multiprocessing:

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from fragmentomics import analyze_sizes
from fragmentomics.io import BamReader

def analyze_single_bam(bam_path):
    """Analyze a single BAM file (for parallel processing)."""
    reader = BamReader(bam_path)
    sizes = reader.extract_sizes()
    dist = analyze_sizes(sizes)
    return dist.to_dict()

# Example parallel processing
# bam_files = list(Path("data/").glob("*.bam"))
# with ProcessPoolExecutor(max_workers=8) as executor:
#     results = list(executor.map(analyze_single_bam, bam_files))

print("For parallel processing:")
print("  with ProcessPoolExecutor(max_workers=8) as executor:")
print("      results = list(executor.map(analyze_single_bam, bam_files))")

## Regional Analysis

Analyze specific genomic regions using a BED file:

In [ ]:
from fragmentomics.io import read_bed, GenomicRegion

# Example: Read promoter regions
# promoters = read_bed_to_list("promoters.bed")
# print(f"Loaded {len(promoters)} regions")

# Analyze fragments in each region
# for region in promoters[:10]:
#     reader = BamReader("sample.bam")
#     sizes = reader.extract_sizes(region=f"{region.chrom}:{region.start}-{region.end}")
#     dist = analyze_sizes(sizes)
#     print(f"{region.name}: {dist.n_fragments} fragments, mean={dist.mean:.1f}")

print("For regional analysis:")
print("  from fragmentomics.io import read_bed_to_list")
print("  regions = read_bed_to_list('promoters.bed')")
print("  for region in regions:")
print("      sizes = reader.extract_sizes(region=region)")

## Tips for Large Datasets

1. **Use streaming** — FragMentor processes BAMs in chunks, not all in memory
2. **Parallel processing** — Use `--threads` flag or `ProcessPoolExecutor`
3. **Index your BAMs** — Ensure `.bai` files exist for fast random access
4. **Filter regions** — Use `--bed` to focus on regions of interest
5. **Save intermediate results** — Use JSON output for resumable analysis